# Setup

In [5]:
# Imports
from pathlib import Path

import polars as pl
import gtfparse
import duckdb

**Remember to review all paths in the Config section below.**

In [6]:
# Paths
project_dir = Path("/data/teamgdansk/mwaleron/carmen-analysis")

data_dir = project_dir.joinpath("data")
temp_dir = project_dir.joinpath("temp")

data_subs_dir = data_dir.joinpath("subsidiary-files")
data_pub_dir = data_dir.joinpath("to-be-published")

gencode_data_dir = data_subs_dir.joinpath("GENCODE")
gencodev39_raw_file = temp_dir.joinpath("gencode.v39.annotation.gtf")
gencodev39_stripped = gencode_data_dir.joinpath("gencode.v39.annotation.stripped.versions.parquet")

contig_scaffold_list_file = data_pub_dir.joinpath("scaff_all_expanded.tsv")
contig_peptide_list_file = data_pub_dir.joinpath("contig_unique_peptide_list.tsv")

exon_regions_file = data_subs_dir.joinpath("exon_regions.parquet")
gene_stats_file = data_subs_dir.joinpath("per_gene_stats.parquet")
exon_regions_per_bp_file = data_subs_dir.joinpath("per_bp_coverage.parquet")

exon_ranges_in_genecode_file = data_subs_dir.joinpath("exon_regions.parquet")
exon_ranges_in_contigs_file = data_subs_dir.joinpath("evenbiggerjoin_cause_its_exons_not_just_genes_v2_just_the_contig_genes.parquet")

# Getting the reference annotation

To begin the analysis we will need to obtain the reference human genome annotation as a GTF file, to replicate the results of this paper you should use the release 39, corresponding to Hg38.p13.

In [7]:
# 
# wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_39/gencode.v39.annotation.gtf.gz
# mv gencode.v39.annotation.gtf.gz temp
# cd temp
# gunzip gencode.v39.annotation.gtf.gz
# cd ...

After obtaining the file we will transform it into a more immediately usable form:

In [8]:
%%script false --no-raise-error
gencodev39_tmp = gtfparse.parse_gtf_and_expand_attributes(gencodev39_raw_file)\
     .with_columns(
     pl.col("gene_id").str.replace(r"\.\d*$", ""),
     pl.col("transcript_id").str.replace(r"\.\d*$", ""),
     pl.col("exon_id").str.replace(r"\.\d*$", ""),
     pl.col("protein_id").str.replace(r"\.\d*$", ""),
 )
gencodev39_tmp.write_parquet(gencodev39_stripped)

In [9]:
gencodev39 = pl.read_parquet(gencodev39_stripped)

The analysis will now need to be sharded to reduce execute time, as such we will create some temporary directories.

In [ ]:
gene_list_file = temp_dir.joinpath("genes.tsv")
temp_dir.joinpath("stats").mkdir(parents=True, exist_ok=True)
temp_dir.joinpath("exons").mkdir(parents=True, exist_ok=True)
temp_dir.joinpath("nonexons").mkdir(parents=True, exist_ok=True)

The easiest way to shard the calculation is to split it by unique gene id:

In [ ]:
pl.DataFrame(gencodev39["gene_id"].unique()).write_csv(gene_list_file, separator="\t")

This is the step where we parallelize the computation, you can use parallel, SLURM, or even run this sequentially.

In [ ]:
#and now you need to run the scripts per gene, have fun
# cat genes.tsv | parallel --bar -j48 python calculate_coverage.py 

Once done we collect the temporary result files

In [4]:
genestats = pl.read_parquet(temp_dir.joinpath("stats/*"))
genestats.write_parquet(gene_stats_file)

In [7]:
exon_regions = pl.read_parquet(temp_dir.joinpath("exons/*"))
exon_regions.write_parquet(exon_regions_file)

Finally we can analyze which parts of the genes are covered by exonic sequences:

In [ ]:
justgenes = gencodev39.filter(pl.col("feature") == "gene")
exon_regions = pl.read_parquet(exon_ranges_in_genecode_file)
cooler_exon_regions = exon_regions.join(justgenes, on="gene_id"
                ).select("gene_id", "exon_ranges", "start", "seqname",
                pl.col("exon_ranges").list.to_array(2).arr.get(0).alias("exon_start") + pl.col("start"),
                pl.col("exon_ranges").list.to_array(2).arr.get(1).alias("exon_end") + pl.col("start")
                ).select(
                    "gene_id", "seqname", "exon_start", "exon_end"
                )

In [ ]:
res = duckdb.sql('''
    select *
    from contig_scaffold_list c, cooler_exon_regions e
    where (e.seqname = c.Chromosome)
        and (
                (c.Gene_start between e.exon_start and e.exon_end)
            or  (c.Gene_end   between e.exon_start and e.exon_end)
            or  (
                    (c.Gene_start < e.exon_start)
                and (c.Gene_end > e.exon_end)
            )
        )
''')
exon_join = res.pl()
exon_join.write_parquet(exon_ranges_in_contigs_file)

# Final Cleanup

This is to clean up and delete all additional files and directories created throughout the analysis.

**Do not run the second cell unless you want to end your work here or start over.**

In [ ]:
# This is a safety code

raise KeyboardInterrupt("Are you sure you want to run the cell below?")

In [ ]:
# Run this cell to do a clean-up
gene_list_file.unlink()
temp_dir.joinpath("stats/*").unlink()
temp_dir.joinpath("stats").rmdir()
temp_dir.joinpath("exons/*").unlink()
temp_dir.joinpath("exons").rmdir()
temp_dir.joinpath("nonexons/*").unlink()
temp_dir.joinpath("nonexons").rmdir()